In [1]:

import uuid
import time
import subprocess
from googleapiclient import discovery
from oauth2client.client import GoogleCredentials
import csv
import sys
import numpy as np
import gzip
import rasterio as rio
import affine
import os.path
from datetime import datetime, timedelta


In [2]:

class BigQuery:

    def __init__(self):
        credentials = GoogleCredentials.get_application_default()
        self._bq = discovery.build('bigquery', 'v2', credentials=credentials)


    # XXX allow dataset/table to be optional (should probably pass in as atomic `destination`)
    # XXXX allow allowLargeResults to be optional
    # XXX check that dataset/table set if not
    # XXX ADD note that if dest table specified it is not automatically deleted
    def async_query(self, project_id, query, dataset, table,
                        batch=False, num_retries=5):
        """Create an asynchronous BigQuery query
        MOAR DOCS
        """
        # Generate a unique job_id so retries
        # don't accidentally duplicate query
        job_data = {
            'jobReference': {
                'projectId': project_id,
                'job_id': str(uuid.uuid4())
            },
            'configuration': {
                'query': {
                    'allowLargeResults': 'true',
                    'writeDisposition':'WRITE_TRUNCATE', #overwrites table
                    'destinationTable' : {
                      "projectId": project_id,
                      "datasetId": dataset,
                      "tableId": table,
                      },
                    'query': query,
                    'priority': 'BATCH' if batch else 'INTERACTIVE'
                }
            }
        }
        return self._bq.jobs().insert(
            projectId=project_id,
            body=job_data).execute(num_retries=num_retries)


    def poll_job(self, job, max_tries=4000):
        """Waits for a job to complete."""

        request = self._bq.jobs().get(
            projectId=job['jobReference']['projectId'],
            jobId=job['jobReference']['jobId'])

        trial = 0
        while trial < max_tries:
            result = request.execute(num_retries=2)

            if result['status']['state'] == 'DONE':
                if 'errorResult' in result['status']:
                    raise RuntimeError(result['status']['errorResult'])
                return

            time.sleep(1)
            trial += 1

        raise RuntimeError("timeout")


    def async_extract_query(self, job, path, format="CSV", compression="GZIP",
                                                        num_retries=5):
        """Extracts query specified by job into Google Cloud storage at path
        MOAR docs
        """

        job_data = {
          'jobReference': {
              'projectId': job['jobReference']['projectId'],
              'jobId': str(uuid.uuid4())
          },
          'configuration': {
              'extract': {
                  'sourceTable': {
                      'projectId': job['configuration']['query']['destinationTable']['projectId'],
                      'datasetId': job['configuration']['query']['destinationTable']['datasetId'],
                      'tableId': job['configuration']['query']['destinationTable']['tableId'],
                  },
                  'destinationUris': [path],
                  'destinationFormat': format,
                  'compression': compression
              }
          }
        }
        return self._bq.jobs().insert(
            projectId=job['jobReference']['projectId'],
            body=job_data).execute(num_retries=num_retries)


def gs_mv(src_path, dest_path):
    """Move data using gsutil
    This was written to move data from cloud
    storage down to your computer and hasn't been
    tested for other things.
    Example:
    gs_mv("gs://world-fishing-827/scratch/SOME_DIR/SOME_FILE",
                "some/local/path/.")
    """
    subprocess.call(["gsutil", "-m", "mv", src_path, dest_path])


# this function is to get the approximate area of a grid cell, 
# which can be used to normalize density. It currently isn't used
def get_area(lat):
    lat_degree = 69 # miles
    # Convert latitude and longitude to 
    # spherical coordinates in radians.
    degrees_to_radians = math.pi/180.0        
    # phi = 90 - latitude
    phi = (lat+cellsize/2.)*degrees_to_radians #plus half a cell size to get the middle
    lon_degree = math.cos(phi)*lat_degree 
    # return 69*69*2.6
    return  lat_degree*lon_degree* 2.58999 # miles to square km




# Execute the query, move the table to gcs, then download it
# as a zipped csv file

proj_id = "world-fishing-827"
dataset = "scratch_global_fishing_raster"


In [3]:

d = datetime(2017,4,7)

for i in range(365):
    # print d + timedelta(days=i)
    thedate = yyyymmdd = datetime.strftime(d + timedelta(days=-i),"%Y%m%d")
#     command = '''python Make_Rasters_201704011.py {yyyymmdd}'''.format(yyyymmdd=yyyymmdd)
#     commands.append(command)
#     thedate = "20150103" 


    path_to_csv_zip = "data/dailytables/fishing_vessel_effort_iso3_gears/zips/"
    path_to_tif =   "data/dailytables/fishing_vessel_effort_iso3_gears/tifs/"
    path_to_csv_zip_iso3 = "data/dailytables/fishing_vessel_effort_iso3_gears/iso3s/"

    # codes = {}
    # all_codes = []
    # with open('iso3.csv','rU') as csvfile:
    #     reader = csv.DictReader(csvfile)
    #     for row in reader:
    #         iso3 = row['iso3']
    #         code = row['code']
    #         if iso3 not in codes:
    #             codes[iso3] = []
    #         codes[iso3].append(code)
    #         all_codes.append(code)


    yyyy = thedate[:4]
    mm = thedate[4:6]
    dd = thedate[6:8]


    query = '''
    SELECT
      FLOOR(lat*10) lat_bin,
      FLOOR(lon*10) lon_bin,
      SUM(hours) hours,
      if(inferred_label is null, "unknown",inferred_label) AS label,
      if(iso3 is null, "UNK", iso3) as iso3
    FROM (
      SELECT
        mmsi,
        lat,
        lon,
        hours,
        flag_iso3 AS iso3
      FROM
        [world-fishing-827:gfw_research.FAO${thedate}]
      WHERE
        measure_new_score > .5 
        and seg_id NOT IN (
        SELECT
          seg_id
        FROM
          [world-fishing-827:gfw_published.segments],
        WHERE
          valid_positions < 10
          AND satellite_positions = 0)) a
    LEFT JOIN (
      SELECT
        mmsi,
        inferred_label
      FROM
        [gfw_research.vessel_info_20170405]
      WHERE
        year={yyyy}) b
    ON
      a.mmsi = b.mmsi
    GROUP BY
      label,
      iso3,
      lat_bin,
      lon_bin
    ORDER BY
      iso3,
      label,
    '''.format(thedate=thedate,yyyy=yyyy)


    table = "fishing_effort_"+thedate
    gcs_path = "gs://david-scratch/"+table+".zip"
    local_path = path_to_csv_zip+table+".zip"
    
    print thedate

    if not os.path.isfile(local_path): 
        bigq = BigQuery()
        query_job = bigq.async_query(proj_id, query, dataset, table)
        bigq.poll_job(query_job)
        extract_job = bigq.async_extract_query(query_job, gcs_path)
        bigq.poll_job(extract_job)
        gs_mv(gcs_path, local_path)

    one_over_cellsize = 10
    cellsize = .1
    num_lats = 180 * one_over_cellsize
    num_lons = 360 * one_over_cellsize


    profile = {
                'crs': 'EPSG:4326',
                'nodata': -9999,
                'dtype': rio.float32,
                'height': num_lats,
                'width': num_lons,
                'count': 6,
                'driver': "GTiff",
                'transform': affine.Affine(float(cellsize), 0, -180, 
                                           0, -float(cellsize), 90),
                'TILED': 'YES',
                'BIGTIFF': 'NO',
                'INTERLEAVE': 'BAND',
                'COMPRESS': 'DEFLATE',
                'PREDICTOR': '3'
            }

    hours = {}
    hours['trawlers'] = np.zeros(shape=(num_lats,num_lons))
    hours['drifting_longlines'] = np.zeros(shape=(num_lats,num_lons))
    hours['purse_seines'] = np.zeros(shape=(num_lats,num_lons))
    hours['fixed_gear'] = np.zeros(shape=(num_lats,num_lons))
    hours['squid_jigger'] = np.zeros(shape=(num_lats,num_lons))
    hours['other_unknown'] = np.zeros(shape=(num_lats,num_lons))
    
    with gzip.open(local_path, 'rb') as f:
        reader = csv.DictReader(f)

        
        n = 0
        for row in reader:
            if n == 0:
                iso3 = row['iso3']
                n = 1
            if row['iso3'] != iso3:  
#                 print hours['trawlers'].sum()

                # # Read bands individually and write individually
                # with rio.open('RGB.byte.tif') as src, \
                #         rio.open('individual.tif', 'w', **src.profile) as dst:
                #     for bidx in range(1, src.count + 1):
                #         data = src.read(bidx)
                #         dst.write(data, indexes=bidx)

                # Kevin says You also want the GeoTIFF creation option `PHOTOMETRIC=MINISBLACK`.  
                # If an image has 3+ bands of type Byte GDAL will assume `PHOTOMETRIC=RGB`
                os.system("mkdir "+path_to_tif + iso3 )
                out_tif = path_to_tif + iso3 +"/"+yyyy+"-"+mm+"-"+dd+".tif"
                with rio.open(out_tif, 'w', **profile) as dst:
                    dst.write(np.flipud(hours['trawlers']).astype(profile['dtype']), indexes=1)
                    dst.write(np.flipud(hours['drifting_longlines']).astype(profile['dtype']), indexes=2)
                    dst.write(np.flipud(hours['purse_seines']).astype(profile['dtype']), indexes=3)
                    dst.write(np.flipud(hours['fixed_gear'] ).astype(profile['dtype']), indexes=4)
                    dst.write(np.flipud(hours['squid_jigger']).astype(profile['dtype']), indexes=5)
                    dst.write(np.flipud(hours['other_unknown']).astype(profile['dtype']), indexes=6)
                    # dst.write((np.flipud(vessel_hours)/2.).astype(profile['dtype']), indexes=2)

                hours = {}
                hours['trawlers'] = np.zeros(shape=(num_lats,num_lons))
                hours['drifting_longlines'] = np.zeros(shape=(num_lats,num_lons))
                hours['purse_seines'] = np.zeros(shape=(num_lats,num_lons))
                hours['fixed_gear'] = np.zeros(shape=(num_lats,num_lons))
                hours['squid_jigger'] = np.zeros(shape=(num_lats,num_lons))
                hours['other_unknown'] = np.zeros(shape=(num_lats,num_lons))
                iso3 = row['iso3']

            lat = int(row['lat_bin'])
            lon = int(row['lon_bin'])   
            if lat<90*one_over_cellsize and \
            lat>-90*one_over_cellsize and lon>-180*one_over_cellsize and lon<180*one_over_cellsize:
                label = row['label']
                if label not in ["drifting_longlines","fixed_gear","squid_jigger", "purse_seines","trawlers"]:
                    label = "other_unknown"
                lat_index = lat+90*one_over_cellsize
                lon_index = lon+180*one_over_cellsize
                hours[label][lat_index][lon_index] = float(row['hours'])
            


20170407
20170406
20170405
20170404
20170403
20170402
20170401
20170331
20170330
20170329
20170328
20170327
20170326
20170325
20170324
20170323
20170322
20170321
20170320
20170319
20170318
20170317
20170316
20170315
20170314
20170313
20170312
20170311
20170310
20170309
20170308
20170307
20170306
20170305
20170304
20170303
20170302
20170301
20170228
20170227
20170226
20170225
20170224
20170223
20170222
20170221
20170220
20170219
20170218
20170217
20170216
20170215
20170214
20170213
20170212
20170211
20170210
20170209
20170208
20170207
20170206
20170205
20170204
20170203
20170202
20170201
20170131
20170130
20170129
20170128
20170127
20170126
20170125
20170124
20170123
20170122
20170121
20170120
20170119
20170118
20170117
20170116
20170115
20170114
20170113
20170112
20170111
20170110
20170109
20170108
20170107
20170106
20170105
20170104
20170103
20170102
20170101
20161231
20161230
20161229
20161228
20161227
20161226
20161225
20161224
20161223
20161222
20161221
20161220
20161219
20161218
2

KeyboardInterrupt: 